# Decoder evaluation

In [1]:
!pip install -U transformers sentencepiece bitsandbytes accelerate rouge_score langchain install -U "ray" --quiet

In [2]:
from huggingface_hub import login
login()

In [3]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

## Evaluator LLM (Llama 2 70b)

In [4]:
from torch import bfloat16
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [5]:
from transformers import AutoConfig

model_id = "meta-llama/Llama-2-70b-chat-hf"

model_config = AutoConfig.from_pretrained(
    model_id,
    token=True
)

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

llama = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    token=True
)

llama_tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=True
)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [7]:
from transformers import pipeline

pipe = pipeline(
    model=llama,
    tokenizer=llama_tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    repetition_penalty=1.1  # without this output begins repeating
)

2023-12-03 18:46:16.241456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 18:46:17.035119: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-03 18:46:17.035283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-03 18:46:17.035290: W tensorfl

In [ ]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

## Decoder Pipeline and evaluation

In [ ]:
from datasets import load_dataset

ds = load_dataset("squad_v2")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
trn = ds["train"]
val = ds["validation"]

In [ ]:
from transformers import Pipeline, TextIteratorStreamer
from threading import Thread

class DecoderPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, text):
        # TODO: Prompt Template -> tokenizer.apply_chat_template
        if self.tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        return self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(self.device)

    def _forward(self, inputs):
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=200)
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()
        generated_text = ""
        for new_text in streamer:
            generated_text += new_text
        generated_text
        return generated_text

    def postprocess(self, outputs):
        outputs = outputs.replace(".\n", " ")
        outputs = outputs.replace("\n", "")
        return outputs

In [ ]:
def get_prompt(question, context):
    return f"""Answer the following Question based on the Context only. Only answer from the Context. If you don't know the answer, say 'I don't know'.
    question: {question}\n\n
    context: {context}\n\n
    Answer:\n"""

In [ ]:
from langchain.evaluation import load_evaluator

qa_evaluator = load_evaluator("context_qa", llm=llm)

def evaluate_context_qa(question, prediction, context, answer):
    res = qa_evaluator.evaluate_strings(
        input=question,
        prediction=prediction,
        context=context,
        reference=answer # Answer
    )
    return res.get("score")

In [ ]:
correctness_evaluator = load_evaluator("labeled_criteria", criteria="correctness", llm=llm, requires_reference=True)

def evaluate_correctness(prompt, prediction, context):
    res = correctness_evaluator.evaluate_strings(
        input=prompt,
        prediction=prediction,
        context=context,
        reference=context # Answer
    )
    return res.get("score"), res.get("reasoning")

In [ ]:
models = [
    "HuggingFaceH4/zephyr-7b-beta",
    "microsoft/Orca-2-13b",
    "mistralai/Mistral-7B-Instruct-v0.1",
    "Intel/neural-chat-7b-v3-1",
    "tiiuae/falcon-7b-instruct",
    "meta-llama/Llama-2-7b-chat-hf",
    #"meta-llama/Llama-2-70b-chat-hf", -> Config
]

In [ ]:
import ray

@ray.remote#(num_gpus=1, max_calls=1)
def evaluate(model_id, iterations, results):
    # Init model
    if "llama" in model_id:
        model_config = AutoConfig.from_pretrained(
            pretrained_model_name_or_path=model_id,
            token=True
        )
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', config=model_config)
        tokenizer = AutoTokenizer.from_pretrained(model_id)
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
    
    decoder = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

    # Test on SQuaDv2
    pbar2 = tqdm(total=iterations, position=1)
    for i in range(0, iterations):
        # Data
        question = val[i]["question"]
        context = val[i]["context"]
        answer_list = val[i]["answers"].get("text")
        answer = "I don't know"
        is_possible = False
        if len(answer_list) > 0:
            is_possible = True
            answer = answer_list[0]
        
        # Inference
        prompt = get_prompt(question=question, context=context)
        pred = decoder(prompt)

        # Evaluation
        qa_score = evaluate_context_qa(
            question=question,
            prediction=pred,
            context=context,
            answer=answer
        )
        correctness_score, reasoning = evaluate_correctness(
            prompt=prompt,
            prediction=pred,
            context=context
        )

        results.append(
            {
                "name": model_id,
                "question": question,
                "prediction": pred,
                "is_possible": is_possible,
                "context": context,
                "qa_score": qa_score,
                "correctness_score": correctness_score,
                "reasoning": reasoning
            }
        )
        
        torch.cuda.empty_cache()
        pbar2.update(1)
    
    # clean up
    del model
    del tokenizer
    del decoder
    
    
    pbar1.update(1)

In [ ]:
from tqdm import tqdm
import pandas as pd
from transformers import logging
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

pbar1 = tqdm(total=len(models), position=0)
iterations = 120

results = []


for model_id in models:
    evaluate.remote(model_id, iterations, results)

  0%|          | 0/6 [00:00<?, ?it/s]2023-12-03 18:46:28,444	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8589930496 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-12-03 18:46:29,630	INFO worker.py:1673 -- Started a local Ray instance.


In [ ]:
df = pd.DataFrame(results)
df.to_pickle("results.pkl")

In [ ]:
del llama
torch.cuda.empty_cache()

In [ ]:
#del model
#del tokenizer
#del decoder
torch.cuda.empty_cache()